In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import scipy
from pprint import pprint
from matplotlib import pyplot as plt
# from inchworm_measurement import simulator
from inchworm_measurement.simulator import Simulator
from scipy.spatial.transform import Rotation
from inchworm_measurement.laser import Laser
from inchworm_measurement import utils

In [2]:
R = Rotation.from_rotvec([np.pi / 12, 0, 0]).as_matrix()
t = np.array([[0, 0, 1000]]).T
T = np.vstack([np.hstack([R, t]), np.array([[0, 0, 0, 1]])])
p2 = np.array([[100, 0, 0], [-100, -0, 0], [0, 100, 0], [0, -100, 0]]).T
# p2 = np.random.rand(3,100)


p1 = utils.homogeneous_transform(T, p2)
pprint(p1)
pprint(p2)

array([[ 100.        , -100.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,   96.59258263,  -96.59258263],
       [1000.        , 1000.        , 1025.88190451,  974.11809549]])
array([[ 100, -100,    0,    0],
       [   0,    0,  100, -100],
       [   0,    0,    0,    0]])


# p1 = T p2を満たすときにp1, p2, tからRを予想したい

In [3]:
T = utils.skew(t)
p1_ = (p1.T @ T).T

A = utils.quadratic2linear(p1, p2.T)
kernel = scipy.linalg.null_space(A)
pprint(kernel)


AttributeError: module 'inchworm_measurement.utils' has no attribute 'skew'

# 先にp1の座標を求めるアプローチ

In [3]:
a = p1[:,0]
b = p1[:,1]
c = p1[:,0]
d = p1[:,1]
t = t[:,0]
pprint([a,b,c])
A = np.hstack([np.cross(t,t).reshape(-1,1),np.cross(a,b).reshape(-1,1), np.cross(a,t).reshape(-1,1),np.cross(b,t).reshape(-1,1)])
B = np.array([np.dot(t,t),np.dot(a,b), np.dot(a,t),np.dot(b,t)])
pprint(A)
pprint(B)

NameError: name 'p1' is not defined

### 3本の直角レーザの原点の位置ベクトルが既知の場合の姿勢推定デモ

In [4]:
R = Rotation.from_rotvec([np.pi / 12, 0, 0]).as_matrix()
t = np.array([[10, 10, 10]]).T
T = np.vstack([np.hstack([R, t]), np.array([[0, 0, 0, 1]])])
p2 = np.array([[100, 0, 0], [0, 100, 0], [0, 0, 100]]).T
p1 = utils.homogeneous_transform(T, p2)

In [6]:
# p1とtからRを推定する
a = np.dot(p1[:, 0], p1[:, 1])
d = np.dot(p1[:, 1], p1[:, 2])
g = np.dot(p1[:, 2], p1[:, 0])
b = -np.dot(p1[:, 0], t[:,0])
e = -np.dot(p1[:, 1], t[:,0])
i = -np.dot(p1[:, 2], t[:,0])
c = e
f = i
h = b
z = np.dot(t[:,0], t[:,0])

A = a * f * i - b * d * i - c * f * g + d * g * z
B = (
    a * f * z
    + a * i * z
    - b * e * i
    - b * d * z
    - c * g * z
    - c * f * h
    + d * h * z
    + e * g * z
)
C = a * z * z - b * e * z - c * h * z + e * h * z
x1 = (-B + np.sqrt(B * B - 4 * A * C)) / (2 * A)
x2 = (-B - np.sqrt(B * B - 4 * A * C)) / (2 * A)
print(x1, x2)

0.17500736581776363 1.0000000000000002


In [7]:
q3 = np.array([x1, x2])
q1 = -1*(i *q3+z)/(g*q3+h)
q2 = -1*(f *q3+z)/(d*q3+e)
pprint([q1, q2, q3])

[array([0.13043478, 1.        ]),
 array([0.10911115, 1.        ]),
 array([0.17500737, 1.        ])]


In [8]:
# 不適な解の詳細を調べる
def xyz_axis_order(rx, ry,rz):
    return np.dot(np.cross(rx, ry), rz)>0
    


p1_a = np.array([[q1[1], q2[1], q3[1]]]).T * p1
p1_b = np.array([[q1[0], q2[0], q3[0]]]).T * p1
p1_a_from_t = p1_a - t
p1_b_from_t =  p1_b - t
p2_a = utils.homogeneous_transform(np.linalg.inv(T), p1_a)
p2_b = utils.homogeneous_transform(np.linalg.inv(T), p1_b)
pprint([p1_a_from_t, p1_b_from_t])

pprint([xyz_axis_order(p1_a_from_t[:,0], p1_a_from_t[:,1], p1_a_from_t[:,2]), xyz_axis_order(p1_b_from_t[:,0], p1_b_from_t[:,1], p1_b_from_t[:,2])])
print(xyz_axis_order(p2[0], p2[1], p2[2]))

[array([[ 1.00000000e+02, -3.55271368e-15, -3.55271368e-15],
       [ 1.77635684e-15,  9.65925826e+01, -2.58819045e+01],
       [ 1.77635684e-15,  2.58819045e+01,  9.65925826e+01]]),
 array([[  4.34782609,  -8.69565217,  -8.69565217],
       [ -8.90888846,   1.63043975, -11.73289294],
       [ -8.24992634,  -3.72040241,   8.6544871 ]])]
[True, False]
True


In [27]:
print(False == False)

True


In [24]:
print(a,b,c,d,e,f,g,h,i,z)
print(A,B,C)

2524.7448713915887 -1300.0 -1524.744871391589 2231.851652578136 -1524.744871391589 -1007.1067811865476 2007.1067811865476 -1300.0 -1007.1067811865476 300
-2099474268.9863536 3384995461.688671 -367423461.41747665


In [14]:
pprint([a,b,c,d,e,f,g,h,i,z])
pprint([A,B,C])

[12588.190451025206,
 -10000.0,
 -12588.190451025206,
 22247.44871391589,
 -12588.190451025206,
 -19659.258262890682,
 19659.258262890682,
 -10000.0,
 -19659.258262890682,
 10000]
[0.0, 2474744871391.589, 0.0]


In [25]:
print(a * f * i, - b * d * i ,- c * f * g , d * g * z)

2560758005.7172074 -2922026814.087996 -3082074836.563159 1343869375.947594


In [20]:
print(a,c,i,g)

32247.44871391589 -32247.44871391589 -27071.067811865476 27071.067811865476


In [41]:
P = np.array(
    [
        [a, b, c, z],
        [0, 0, d + e, f + z],
        [0, g + h, 0, i + z],
        [d + e, f + z, 0, 0],
    ]
)
print(P)
print(np.linalg.matrix_rank(P))

[[ 2524.74487139 -1300.         -1524.74487139   300.        ]
 [    0.             0.           707.10678119  -707.10678119]
 [    0.           707.10678119     0.          -707.10678119]
 [  707.10678119  -707.10678119     0.             0.        ]]
3


In [47]:
q=1
P = np.array(
    [
        [a, b, c, z],
        [0, 0, d*q + e, f*q + z],
        [0, g*q + h, 0, i*q + z],
        [d*q + e, f*q + z, 0, 0],
    ]
)
print(P)
print(np.linalg.matrix_rank(P))

[[ 2524.74487139 -1300.         -1524.74487139   300.        ]
 [    0.             0.           707.10678119  -707.10678119]
 [    0.           707.10678119     0.          -707.10678119]
 [  707.10678119  -707.10678119     0.             0.        ]]
3


In [132]:
P1 = np.array(
    [
        [0, d * q + e, f * q + z],
        [g * q + h, 0, i * q + z],
        [f * q + z, 0, 0],
    ]
)
P2 = np.array(
    [
        [0, d * q + e, f * q + z],
        [0, 0, i * q + z],
        [d * q + e, 0, 0],
    ]
)
P3 = np.array(
    [
        [0, 0, f * q + z],
        [0, g * q + h, i * q + z],
        [d * q + e, f * q + z, 0],
    ]
)
P4 = np.array(
    [
        [0, 0, d * q + e],
        [0, g * q + h, 0],
        [d * q + e, f * q + z, 0],
    ]
)

print(
    a * np.linalg.det(P1)
    - b * np.linalg.det(P2)
    + c * np.linalg.det(P3)
    - z * np.linalg.det(P4)
)

NameError: name 'd' is not defined

3点アルゴリズムのモジュール化

点の位置を未知数にするアプローチ

In [18]:
def sfm_by_orthogonal_three_points(c1_P_dir, c1_t_c2):
    print(c1_P_dir)
    print(c1_t_c2)
    # C1からの方向ベクトルと並進ベクトルからスケールを決定する
    a = np.dot(c1_P_dir[:, 0], c1_P_dir[:, 1])
    b = np.dot(c1_P_dir[:, 1], c1_P_dir[:, 2])
    c = np.dot(c1_P_dir[:, 2], c1_P_dir[:, 0])
    d = -np.dot(c1_P_dir[:, 0], c1_t_c2[:, 0])
    e = -np.dot(c1_P_dir[:, 1], c1_t_c2[:, 0])
    f = -np.dot(c1_P_dir[:, 2], c1_t_c2[:, 0])
    g = np.dot(c1_t_c2[:, 0], c1_t_c2[:, 0])

    Q_candiate = []
    if len(Q_candiate) == 0:
        # q1についての2次方程式を解く(ax+d)(Ax^2+Bx+C) = 0
        A = c*e*e - a*f*e + a*b*g - d*b*e
        B = 2*(c*e*g - f*d*e)
        C = c*g*g - f*d*g
        print(a,d,A,B,C)
        if (a != 0 or d != 0) and (A != 0 or B != 0 or C != 0):  # (ax+d)(Ax^2+Bx+C) = 0 が自明でない場合
            x = []
            if A != 0:
                x.append((-B + np.sqrt(B * B - 4 * A * C)) / (2 * A))
                x.append((-B - np.sqrt(B * B - 4 * A * C)) / (2 * A))
            elif B != 0:
                x.append(-C/B)
            if a != 0:
                x.append(-d/a)
            x = list(set(x))  # 重複を削除
            for q1 in x:
                if (a*q1 + e == 0 or c*q1 + f == 0):
                    continue
                q2 = -(d*q1 + g)/(a*q1 + e)
                q3 = -(d*q1 + g)/(c*q1 + f)
                Q_candiate.append(np.array([q1, q2, q3]))

    # if len(Q_candiate) == 0:
        # q2についての2次方程式を解く(cx+f)(Ax^2+Bx+C) = 0
        A = b*d*d - c*e*d + c*a*g - f*a*d
        B = 2*(b*d*g - e*f*d)
        C = b*g*g - e*f*g
        print(c, f, A, B, C)
        if (c != 0 or f != 0) and (A != 0 or B != 0 or C != 0):  # (cx+f)(Ax^2+Bx+C) = 0 が自明でない場合
            x = []
            print(x)
            if A != 0:
                x.append((-B + np.sqrt(B * B - 4 * A * C)) / (2 * A))
                x.append((-B - np.sqrt(B * B - 4 * A * C)) / (2 * A))
                print((-B + np.sqrt(B * B - 4 * A * C)) / (2 * A))
            elif B != 0:
                print(B,C,C/B)
                x.append(-C/B)
            if c != 0:
                x.append(-f/c)
            print("x",x)
            x = list(set(x))  # 重複を削除
            for q2 in x:
                if (b*q2 + f == 0 or a*q2 + d == 0):
                    continue
                q3 = -(e*q2 + g)/(b*q2 + f)
                q1 = -(e*q2 + g)/(a*q2 + d)
                Q_candiate.append(np.array([q1, q2, q3]))

    # if len(Q_candiate) == 0:
        # q3についての2次方程式を解く(bx+e)(Ax^2+Bx+C) = 0
        A = a*f*f - b*d*f + b*c*g - e*c*f
        B = 2*(a*f*g - d*e*f)
        C = a*g*g - d*e*g
        print(b, e, A, B, C)
        if (b != 0 or e != 0) and (A != 0 or B != 0 or C != 0):  # (bx+e)(Ax^2+Bx+C) = 0 が自明でない場合
            x = []
            if A != 0:
                x.append((-B + np.sqrt(B * B - 4 * A * C)) / (2 * A))
                x.append((-B - np.sqrt(B * B - 4 * A * C)) / (2 * A))
            elif B != 0:
                x.append(-C/B)
            if b != 0:
                x.append(-e/b)
            x = list(set(x))  # 重複を削除
            for q3 in x:
                if (c*q3 + d == 0 or b*q3 + e == 0):
                    continue
                q1 = -(f*q3 + g)/(c*q3 + d)
                q2 = -(f*q3 + g)/(b*q3 + e)
                Q_candiate.append(np.array([q1, q2, q3]))

    score = []
    result_candidate = []
    print(Q_candiate)
    for Q in Q_candiate:
        Q = np.array([1900, 3900, 3900])
        
        c1_P = Q * c1_P_dir
        print(c1_P)
        c1_P_T = c1_P - c1_t_c2
        # 十分性チェック
        cos12 = np.abs(np.dot(c1_P_T[:, 0], c1_P_T[:, 1]) / (np.linalg.norm(c1_P_T[:, 0]) * np.linalg.norm(c1_P_T[:, 1])))
        cos23 = np.abs(np.dot(c1_P_T[:, 1], c1_P_T[:, 2]) / (np.linalg.norm(c1_P_T[:, 1]) * np.linalg.norm(c1_P_T[:, 2])))
        cos31 = np.abs(np.dot(c1_P_T[:, 2], c1_P_T[:, 0]) / (np.linalg.norm(c1_P_T[:, 2]) * np.linalg.norm(c1_P_T[:, 0])))
        score.append(cos12 + cos23 + cos31)
    
    print(score)
    min_index = int(np.array(score).argmin())
    c1_P = Q_candiate[min_index] * c1_P_dir    
    c1_P_T = c1_P - c1_t_c2
    c1_R_c2 = c1_P_T / np.linalg.norm(c1_P_T, axis=0)
    c1_M_c2 = utils.pose_Rt2M(c1_R_c2, c1_t_c2)
    c2_P = utils.homogeneous_transform(np.linalg.inv(c1_M_c2), c1_P)
    return [c1_P, c2_P, c1_M_c2]


R = Rotation.from_rotvec([0, 0, 0]).as_matrix()
t = np.array([[30, 0, 10]]).T
# R = np.eye(3)
# t = np.array([[0, 0, 1]]).T
T = np.vstack([np.hstack([R, t]), np.array([[0, 0, 0, 1]])])

p1 = np.array([[-1.0525725,   0.,          0.],
               [0.,         0.51283875, - 0.51283875],
               [1.,        1.,          1.]])
t = np.array([[0.],
              [0.],
              [1900.]])
# print(T)
# p2 = np.array([[10, 0, 0], [0, 500, 0], [0, 0, 100]]).T
# p1 = utils.homogeneous_transform(T, p2)


[c1_p_est, c2_p_est, R_est] = sfm_by_orthogonal_three_points(p1, t)
# [c1_p_est, c2_p_est, R_est] = utils.sfm_by_orthogonal_three_points(p1, t)

print("c1_p_est", c1_p_est)
print("c2_p_est", c2_p_est)
print("R_est", R_est)


[[-1.0525725   0.          0.        ]
 [ 0.          0.51283875 -0.51283875]
 [ 1.          1.          1.        ]]
[[   0.]
 [   0.]
 [1900.]]
1.0 -1900.0 0.0 0.0 0.0
1.0 -1900.0 -949442.9364406406 3607883158.474434 -3427489000550.713
[]
nan
x [nan, nan, 1900.0]
0.7369964164984375 -1900.0 0.0 0.0 0.0
[array([nan, nan, nan]), array([nan, nan, nan])]
[[-1999.88775      0.           0.      ]
 [    0.        2000.071125 -2000.071125]
 [ 1900.        3900.        3900.      ]]
[[-1999.88775      0.           0.      ]
 [    0.        2000.071125 -2000.071125]
 [ 1900.        3900.        3900.      ]]
[3.556186765427057e-05, 3.556186765427057e-05]


/var/folders/xt/02mw1qr55355x_tfz9rlb9z00000gn/T/ipykernel_83790/3269927432.py:47: RuntimeWarning: invalid value encountered in sqrt
  x.append((-B + np.sqrt(B * B - 4 * A * C)) / (2 * A))
/var/folders/xt/02mw1qr55355x_tfz9rlb9z00000gn/T/ipykernel_83790/3269927432.py:48: RuntimeWarning: invalid value encountered in sqrt
  x.append((-B - np.sqrt(B * B - 4 * A * C)) / (2 * A))
/var/folders/xt/02mw1qr55355x_tfz9rlb9z00000gn/T/ipykernel_83790/3269927432.py:49: RuntimeWarning: invalid value encountered in sqrt
  print((-B + np.sqrt(B * B - 4 * A * C)) / (2 * A))


LinAlgError: Singular matrix

In [34]:
p1 = np.array([[-1.0525725,   0.,          0.],
               [0.,         0.51283875, - 0.51283875],
               [1.,        1.,          1.]])
q1 = 2000/1.0525725
q2 = 2000/51283875
q3 = q2

1

エピポーラベースのアプローチ

In [128]:
def cross_to_matrix(v):
    assert v.shape == (3, 1), "invalid shape"
    return np.cross(v[:, 0], -np.identity(3))


def three_points_algorithm(c1_P_dir, c1_t_c2, R):
    c2_P_dir = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).T
    # C1からの方向ベクトルと並進ベクトルからスケールを決定する
    t = utils.vect2col(c1_t_c2[:, 0])
    p1 = utils.vect2col(c1_P_dir[:, 0])
    p2 = utils.vect2col(c1_P_dir[:, 1])
    p3 = utils.vect2col(c1_P_dir[:, 2])

    r1 = utils.vect2col(R[:, 0])
    r2 = utils.vect2col(R[:, 1])
    r3 = utils.vect2col(R[:, 2])

    u1 = (p1.T @ cross_to_matrix(t)).T
    u2 = (p2.T @ cross_to_matrix(t)).T
    u3 = (p3.T @ cross_to_matrix(t)).T

    U1 = cross_to_matrix(u1)
    U2 = cross_to_matrix(u2)
    U3 = cross_to_matrix(u3)

    # print([u1, r2, r3])
    # print([u2, r1, r2])
    # print([u3, r3, r1])

    print("r1", r1)
    print(np.linalg.norm(r1))

    # v = np.linalg.eig(U1@U2)[1][0:1,:].T
    # V = np.linalg.eig(U1@U2)[1]
    # V_norm = np.linalg.norm(V, axis=0)
    # v = V[:,0:1]
    # print(V)
    # print(V_norm)
    # print(v)
    # print(U1@U2 @ v)
    # [l, v]  =  np.linalg.eig(U1@U2)
    # v1 = utils.vect2col(v[:,0])
    # v2 = utils.vect2col(v[:,1])
    # v3 = utils.vect2col(v[:,2])

    [l1, v1] = np.linalg.eig(U1@U2)
    [l2, v2] = np.linalg.eig(U1@U3)

    print(l1, l2)
    print(v1, v2)
    # print(l2)
    # print(v2)
    


    A1 = U1@U2-l1[0]*np.identity(3)
    A2 = U1@U3-l2[0]*np.identity(3)
    A12 = np.vstack([A1, A2])
    print(A12)
    print("rank", np.linalg.matrix_rank(A12))
    print(A)
    print(A12@utils.vect2col(v1[:,0]))
    print(A12@utils.vect2col(v2[:,0]))
    print(A12@r1)
    print(np.linalg.matrix_rank(np.vstack([A12, u1.T])))

    # print(A2)

    # v1_ = U1@U2@v1
    # v2_ = U1@U2@v2
    # v3_ = U1@U2@v3
    # r1_ = U1@U2@r1

    # v1_norm = np.linalg.norm(v1_) / np.linalg.norm(v1)
    # v2_norm = np.linalg.norm(v2_) / np.linalg.norm(v2)
    # v3_norm = np.linalg.norm(v3_) / np.linalg.norm(v3)
    # r1_norm = np.linalg.norm(r1_) / np.linalg.norm(r1)
    # print(np.hstack([v1, v2, v3, r1]))
    # print([v1_norm, v2_norm, v3_norm, r1_norm])

    # print(U1@U2 + 114.94496960738734*np.identity(3))
    # print( np.linalg.eig(U1@U3)[1])
    # print( np.linalg.eig(U2@U1)[1])
    # print( np.linalg.eig(U2@U3)[1])
    # print( np.linalg.eig(U3@U1)[1])
    # print( np.linalg.eig(U3@U2)[1])

    # print(Q_candiate)
    # for Q in Q_candiate:
    #     c1_P = Q * c1_P_dir
    #     c2_P = c1_P - c1_t_c2
    #     # validation
    #     v1 = c2_P[:,1] - c2_P[:,0]
    #     v2 = c2_P[:,2] - c2_P[:,0]
    #     abs_cos = np.abs(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    #     if abs_cos < 0.001:
    #         continue
    #     print("p1", c1_P)
    #     print("p2", c2_P)
    #     c1_R_c2 = c2_P / np.linalg.norm(c2_P, axis=0)

    # return [c1_P, c1_R_c2]

    # # 不適な解の除外
    # for
    # q3 = np.array([x_a, x_b])
    # q1 = -1*(i * q3+z)/(g*q3+h)
    # q2 = -1*(f * q3+z)/(d*q3+e)
    # c1_P_a = np.array([[q1[0], q2[0], q3[0]]]) * c1_P_dir
    # c1_P_b = np.array([[q1[1], q2[1], q3[1]]]) * c1_P_dir
    # c1_P = c1_P_a if q1[0] > 0 and q2[0] > 0 and q3[0] > 0 else c1_P_b
    # Q_candiate = c1_P - t
    # c1_R_c2 = Q_candiate / np.linalg.norm(Q_candiate, axis=0)

    # return [c1_P, c1_R_c2]


R = Rotation.from_rotvec([0.2, 0.3, 0.3]).as_matrix()
t = np.array([[30, 20, 10]]).T
# R = np.eye(3)
# t = np.array([[0, 0, 1]]).T
T = np.vstack([np.hstack([R, t]), np.array([[0, 0, 0, 1]])])
print(T)
p2 = np.array([[10, 0, 0], [0, 500, 0], [0, 0, 100]]).T
p1 = utils.homogeneous_transform(T, p2)

# p1[:,0] = p1[:,0] / np.linalg.norm(p1[:,0])
# p1[:,1] = p1[:,1] / np.linalg.norm(p1[:,1])
# p1[:,2] = p1[:,2] / np.linalg.norm(p1[:,2])
# print(p1)
# print(R)
[p1_est, R_est] = three_points_algorithm(p1/10, t, R)


[[ 0.91163795 -0.25966635  0.31857439 30.        ]
 [ 0.31857439  0.93618296 -0.14856589 20.        ]
 [-0.25966635  0.23692794  0.93618296 10.        ]
 [ 0.          0.          0.          1.        ]]
r1 [[ 0.91163795]
 [ 0.31857439]
 [-0.25966635]]
1.0
[ 2.07020230e+04 -2.23953629e-13  2.07020230e+04] [ 4.96356972e+03 -6.84745722e-13  4.96356972e+03]
[[ 0.84169599 -0.13219971  0.21539753]
 [ 0.15116036 -0.27749452  0.52822193]
 [-0.51836127  0.95158816  0.82133154]] [[ 0.82088335  0.59706552  0.46264976]
 [ 0.52371581 -0.73563982  0.57539083]
 [ 0.22775486 -0.31991691  0.67444836]]
[[  1936.93570598  -3908.13638056   2005.46564318]
 [  4065.7353564   -8203.39478022   4209.58349125]
 [-13942.27783073  28131.19870212 -14435.56391203]
 [ -1693.35308549   3416.6620906   -1753.26492472]
 [  2086.36727997  -4209.64301751   2160.18419512]
 [   907.32469655  -1830.70023678    939.42638391]]
rank 1
[[  1936.93570598  -3908.13638056   2005.46564318]
 [  4065.7353564   -8203.39478022   4209.

TypeError: cannot unpack non-iterable NoneType object

In [106]:
A = np.array([[1936.93570598, - 3908.13638056,   2005.46564318],
              [4065.7353564, - 8203.39478022,  4209.58349125],
              [-13942.27783073, 28131.19870212, - 14435.56391203]]
             )
np.linalg.solve(A, np.array([0, 0, 0]))


array([-0.,  0.,  0.])

3点法テスト

In [19]:
R = Rotation.from_rotvec([0.1, 0.1, 0.2]).as_matrix()
t = np.array([[5, 20, 10]]).T

print(R)
# R = np.eye(3)
# t = np.array([[0, 0, 1]]).T
T = np.vstack([np.hstack([R, t]), np.array([[0, 0, 0, 1]])])
p2 = np.array([[10, 0, 0], [0, 500, 0], [0, 0, 100]]).T
p1 = utils.homogeneous_transform(T, p2)

# p1[:,0] = p1[:,0] / np.linalg.norm(p1[:,0])
# p1[:,1] = p1[:,1] / np.linalg.norm(p1[:,1])
# p1[:,2] = p1[:,2] / np.linalg.norm(p1[:,2])
# print(p1)
# print(R)
[p1_est, R_est] = three_points_algorithm(p1/10, t)
# pprint(P1)
# pprint(P2)
print("--------------------")
print(p1)
print(R)
print(p1_est)
print(R_est)

[[ 0.97512475 -0.19303094  0.1089531 ]
 [ 0.20298104  0.97512475 -0.0890529 ]
 [-0.0890529   0.1089531   0.9900499 ]]
[[ 0.98326332 -0.64045634 -0.44925621]
 [ 0.0496619  -0.20934212  0.73991553]
 [-0.17529104  0.73891241  0.50069338]]
[[ 0.98326332]
 [ 0.0496619 ]
 [-0.17529104]]
[[ 7762.49995581]
 [  392.06235929]
 [-1383.85787334]]


NameError: name 'Q_candiate' is not defined

3点法モジュール線形バージョン（ボツ）

In [105]:
def three_points_algorithm(c1_P_dir, c1_t_c2):
    c2_P_dir = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).T
    # C1からの方向ベクトルと並進ベクトルからスケールを決定する
    a = np.dot(c1_P_dir[:, 0], c1_P_dir[:, 1])
    b = np.dot(c1_P_dir[:, 1], c1_P_dir[:, 2])
    c = np.dot(c1_P_dir[:, 2], c1_P_dir[:, 0])
    d = -np.dot(c1_P_dir[:, 0], c1_t_c2[:, 0])
    e = -np.dot(c1_P_dir[:, 1], c1_t_c2[:, 0])
    f = -np.dot(c1_P_dir[:, 2], c1_t_c2[:, 0])
    g = np.dot(c1_t_c2[:, 0], c1_t_c2[:, 0])

    # A = np.array([
    #     [a, 0, 0, d, e, 0],
    #     [0, b, 0, 0, e, f],
    #     [0, 0, c, d, 0, f],
    #     [-a*c*e, b*c*e, b*c*d-a*c*f, -a*c*g, 0, b*c*g],
    #     [a*c*e-a*b*d, -a*b*f, a*c*f, a*c*g, -a*b*g, 0],
    #     [a*b*d, a*b*f-b*c*e, -b*c*d, 0, a*b*g, -b*c*g],
    # ])
    # B = -np.array([[g, g, g, 0, 0, 0]]).T

    # A = np.array([
    #     [-a*c*g + a*f*d - b*d*d + c*e*d, 0, a*f*f + b*c*g - b*d*f - c*e*f],
    #     [b*d*d + c*a*g - c*e*d - a*f*d, -b*a*g + b*d*e - c*e*e + a*f*e, 0],
    #     [0, c*e*e + a*b*g - a*f*e - b*d*e, -c*b*g + c*e*f - a*f*f + b*d*f],
    # ])
    # B= np.array([
    #     [a*f*g - b*d*g],
    #     [b*d*g - c*e*g],
    #     [c*e*g - a*f*g],
    # ])
    # print("A", A)

    # x = np.linalg.inv(A) @ B
    # x_ = np.linalg.solve(A, B)
    # print(np.linalg.inv(A))
    # print(A@x)
    # print(A @ np.array([
    #     [1, 1, 1],
    # ]).T)

    # # kenzan_true = np.array([
    # #     [a, 0, 0, d, e, 0, g],
    # #     [0, b, 0, 0, e, f, g],
    # #     [0, 0, c, d, 0, f, g],
    # # ]) @ np.array([
    # #     [1, 1, 1, 1, 1, 1, 1],
    # # ]).T

    # # kenzan_est = np.array([
    # #     [a, 0, 0, d, e, 0, g],
    # #     [0, b, 0, 0, e, f, g],
    # #     [0, 0, c, d, 0, f, g],
    # # ]) @ np.vstack([x,1])
    # # print("kenzan_true", kenzan_true)
    # # print("kenzan_est", kenzan_est)

    # print("q", x)
    # print("q_", x_)
    # # [q1q2, q2q3, q3q1, q1, q2, q3]= x
    # [q1, q2, q3] = x
    # c1_P = np.array([[q1, q2, q3]]) * c1_P_dir
    # Q = c1_P - c1_t_c2
    # c1_R_c2 = Q / np.linalg.norm(Q, axis=0)
    # return [c1_P, c1_R_c2]

    # 不適な解の除外
    # q3 = np.array([x_a, x_b])
    # q1 = -1*(i *q3+z)/(g*q3+h)
    # q2 = -1*(f *q3+z)/(d*q3+e)
    # c1_P_a = np.array([[q1[0], q2[0], q3[0]]]) * c1_P_dir
    # c1_P_b = np.array([[q1[1], q2[1], q3[1]]]) * c1_P_dir
    # c1_P = c1_P_a if q1[0] > 0 and q2[0] > 0 and q3[0] > 0 else c1_P_b
    # Q = c1_P - t
    # c1_R_c2 = Q / np.linalg.norm(Q, axis=0)

    # return [c1_P, c1_R_c2]


### 5点法テスト

In [43]:
R = Rotation.from_rotvec([10, 50, 0]).as_matrix()
t = np.array([[10, 100, 1000]]).T
T = np.vstack([np.hstack([R, t]), np.array([[0, 0, 0, 1]])])
n = 6
p2 = np.array(
        [[np.cos(i * np.pi / n * 2), np.sin(i * np.pi / n * 2), 1] for i in range(n)]
    ).T
p1 = utils.homogeneous_transform(T, p2)

pprint(p1.T)
pprint(R)
[P1, P2, M]  = utils.sfm_by_five_points(p1, p2)
pprint(P1.T)
pprint(P2.T)
pprint(M)


array([[  11.40840259,   99.91831948, 1000.09864303],
       [  11.07107876,  100.75180965, 1000.5362579 ],
       [  10.31272306,  100.70348079, 1001.18630479],
       [   9.89169119,   99.82166176, 1001.39873681],
       [  10.22901502,   98.98817159, 1000.96112195],
       [  10.98737072,   99.03650045, 1000.31107506]])
array([[ 0.7583557 ,  0.04832886,  0.65004689],
       [ 0.04832886,  0.99033423, -0.13000938],
       [-0.65004689,  0.13000938,  0.74868992]])
array([[0.01135122, 0.09941752, 0.99508608],
       [0.01101559, 0.10024683, 0.9955215 ],
       [0.01026104, 0.10019875, 0.99616829],
       [0.00984211, 0.09932135, 0.99637966],
       [0.01017775, 0.09849204, 0.99594424],
       [0.0109323 , 0.09854012, 0.99529745]])
array([[ 9.94987935e-04,  0.00000000e+00,  9.94987935e-04],
       [ 4.97493967e-04,  8.61684828e-04,  9.94987935e-04],
       [-4.97493967e-04,  8.61684828e-04,  9.94987935e-04],
       [-9.94987935e-04,  1.21850879e-19,  9.94987935e-04],
       [-4.97493967

In [32]:
Points

6

In [8]:
from sympy import *
# 曲面を方程式で定義
x, y, z = symbols('x y z')
f = x**2 + y**2 + z**2 - 5
# 直線を点と方向ベクトルで定義
p0 = Point3D(0, 0, 0)
v0 = Point3D(1, 1, 1)
l = Line3D(p0, direction_ratio=v0)
# 交点を求める
# intersection = l.intersection(f)
c = Curve
result = intersection(l, l)
print(result)

[Line3D(Point3D(0, 0, 0), Point3D(1, 1, 1))]


In [14]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import sympy as sp

# 半直線と陰関数の交点を出す関数
def intersection_point(v, f):
  # 方向ベクトルと陰関数から半直線と曲面の方程式を作る
  x, y, z, t = sp.symbols('x y z t')
  a, b, c = v
  r = sp.Matrix([t*a, t*b, t*c])
  surface = sp.Eq(f(x, y, z), 0)
  # 方程式を連立して解く
  sol = sp.solve([surface.subs({x:r[0], y:r[1], z:r[2]}), t >= 0], t)
  sol = list(sol) # ここを追加
  print(sol)
  # 解が存在する場合は、交点の座標を返す
  if True:
    t_value = sol[0][0]
    point = r.subs(t, t_value)
    return point
  else:
    return None

# 方向ベクトルと陰関数の例
v = (1, 2, 3)
f = lambda x, y, z: x**2 + y**2 + z**2 - 25

# 交点を出す関数を呼び出す
point = intersection_point(v, f)
if point:
  print(f"交点は{point}です。")
else:
  print("交点はありません。")


TypeError: 'Equality' object is not iterable

In [45]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import sympy as sp

# 半直線と陰関数の交点を出す関数
def intersection_point(v, f_list):
    # 方向ベクトルと陰関数から半直線と曲面の方程式を作る
    t = sp.symbols('t')
    a, b, c = v
    eq_list = [f(t*a, t*b, t*c) for f in f_list]
    eq_list.append(t >= 0)

    #   sol = sp.solve(sp.solve([eq1, t >= 0],),t)
    sol = sp.solve(sp.solve(eq_list),t)

    #   sol = sp.solve(sol, t, dict=True)
    print(sol)
    # 解が存在する場合は、交点の座標を返す
    if sol:
        t_value = sol[0]
        return t_value * np.array(v)
    else:   
        return None

# 方向ベクトルと陰関数の例
v = (1, 2, 3)
f = [lambda x, y, z: x**2 + y**2 + z**2 - 25]

# 交点を出す関数を呼び出す
point = intersection_point(v, f)
if len(point):
  print(f"交点は{point}です。")
else:
  print("交点はありません。")


[5*sqrt(14)/14]
交点は[5*sqrt(14)/14 5*sqrt(14)/7 15*sqrt(14)/14]です。


In [35]:
f = [lambda x, y, z: x**2 + y**2 + z**2 - 25, lambda x, y, z: x**2 + y**2 + z**2 - 25]

In [38]:
g = [f_ for f_ in f]

TypeError: Too many arguments for numpy.ndarray[typing.Any, numpy.dtype[+ScalarType]]

In [56]:
np.array([None, None , None])

array([None, None, None], dtype=object)